In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import pandas as pd
from numpy import asarray
from numpy import save
from matplotlib import pyplot as plt



In [2]:
interactions_from_selected_users_desc_df = pd.read_csv('/Users/jocelynpok/Downloads/recommender/interactions_from_selected_users_desc_df.csv', index_col='Unnamed: 0')

In [3]:
users_items_pivot_matrix_df = interactions_from_selected_users_desc_df.pivot(index = 'reviewerID', columns ='asin', values = 'overall')

The PMF class is instantiated with 10 latent factors, precision value of 2, standard deviation of 0.01, and a lower and upper bound of 1 and 5 respectively.

In [4]:
import time
import logging
import theano
import scipy as sp

theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
# Enable on-the-fly graph computations, but ignore
# absence of intermediate test values.
theano.config.compute_test_value = 'ignore'

# Set up logging.
logger = logging.getLogger()
logger.setLevel(logging.INFO)


class PMF():
    """Probabilistic Matrix Factorization model using pymc3."""

    def __init__(self, train, dim, alpha=2, std=0.01, bounds=(1, 5)):
        """Build the Probabilistic Matrix Factorization model using pymc3.

        :param np.ndarray train: The training data to use for learning the model.
        :param int dim: Dimensionality of the model; number of latent factors.
        :param int alpha: Fixed precision for the likelihood function.
        :param float std: Amount of noise to use for model initialization.
        :param (tuple of int) bounds: (lower, upper) bound of ratings.
            These bounds will simply be used to cap the estimates produced for R.

        """
        self.dim = dim
        self.alpha = alpha
        self.std = np.sqrt(1.0 / alpha)
        self.bounds = bounds
        self.data = train.copy()
        n, m = self.data.shape

        # Perform mean value imputation
        nan_mask = np.isnan(self.data)
        self.data[nan_mask] = self.data[~nan_mask].mean()

        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = 1 / self.data.var(axis=1).mean()
        self.alpha_v = 1 / self.data.var(axis=0).mean()

        # Specify the model.
        logging.info('building the PMF model')
        with pm.Model() as pmf:
            U = pm.MvNormal(
                'U', mu=0, tau=self.alpha_u * np.eye(dim),
                shape=(n, dim), testval=np.random.randn(n, dim) * std)
            V = pm.MvNormal(
                'V', mu=0, tau=self.alpha_v * np.eye(dim),
                shape=(m, dim), testval=np.random.randn(m, dim) * std)
            R = pm.Normal(
                'R', mu=(U @ V.T)[~nan_mask], tau=self.alpha,
                observed=self.data[~nan_mask])

        logging.info('done building the PMF model')
        self.model = pmf

    def __str__(self):
        return self.name

The maximum a posteriori (MAP) estimate can be derived using the built-in utilities in pymc3.

In [5]:
def _find_map(self):
    """Find mode of posterior using L-BFGS-B optimization."""
    tstart = time.time()
    with self.model:
        logging.info('finding PMF MAP using L-BFGS-B optimization...')
        self._map = pm.find_MAP(method='L-BFGS-B')

    elapsed = int(time.time() - tstart)
    logging.info('found PMF MAP in %d seconds' % elapsed)
    return self._map

def _map(self):
    try:
        return self._map
    except:
        return self.find_map()

# Update our class with the new MAP infrastructure.
PMF.find_map = _find_map
PMF.map = property(_map)

To generate predictions from the sampler, we generate an 𝑅
matrix for each 𝑈 and 𝑉 sampled, then we combine these by averaging over the 𝐾 samples.

In [7]:
def _predict(self, U, V):
    """Estimate R from the given values of U and V."""
    R = np.dot(U, V.T)
    n, m = R.shape
    sample_R = np.random.normal(R, self.std)
    # bound ratings
    low, high = self.bounds
    sample_R[sample_R < low] = low
    sample_R[sample_R > high] = high
    return sample_R


PMF.predict = _predict

In [12]:
# We use a fixed precision for the likelihood.
# This reflects uncertainty in the dot product.
# We choose 2 in the footsteps Salakhutdinov
# Mnihof.
ALPHA = 2

# The dimensionality D; the number of latent factors.
# We can adjust this higher to try to capture more subtle
# characteristics of each movie. However, the higher it is,
# the more expensive our inference procedures will be.

DIM = 10


#pmf = PMF(train, DIM, ALPHA, std=0.05)
pmf = PMF(users_items_pivot_matrix_df.values, DIM, ALPHA, std=0.05)

INFO:root:building the PMF model
INFO:root:done building the PMF model


In [13]:
# Find MAP for PMF.
pmf.find_map();

INFO:root:finding PMF MAP using L-BFGS-B optimization...
logp = -6.4204e+06, ||grad|| = 60,095: 100%|██████████| 8/8 [01:05<00:00,  8.20s/it]      
INFO:root:found PMF MAP in 106 seconds


In [ ]:
def get_pred(pmf_model):
    U = pmf_model.map['U']
    V = pmf_model.map['V']

    # Make predictions 
    predictions = pmf_model.predict(U, V)
   
    return predictions


# Add eval function to PMF class.
PMF.get_pred = get_pred

In [ ]:
map_preds = pmf.get_pred()

In [ ]:

# save to npy file
save('/Users/jocelynpok/Downloads/recommender/map_preds.npy', map_preds)